In [1]:
import torch
import h5py
import os
import numpy as np
import sys
from tqdm import tqdm
sys.path.append('generative_models/')
import sgm
from generative_models.sgm.modules.encoders.modules import FrozenOpenCLIPImageEmbedder
from torch.utils.data import Dataset

# from generative_models.sgm.models.diffusion import DiffusionEngine
# from omegaconf import OmegaConf

/admin/home-mihirneal/miniconda3/envs/mindeye/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/admin/home-mihirneal/miniconda3/envs/mindeye/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
d1 = torch.device("cuda:0")
d2 = torch.device("cuda:1")
d3 = torch.device("cuda:2")

In [17]:
data_path = "/weka/proj-fmri/shared/mindeyev2_dataset"
subj=1
data_type = torch.float32
num_sessions=3
num_devices=3
batch_size = 8
import webdataset as wds
import random

In [18]:
def my_split_by_node(urls): return urls

nsessions_allsubj=np.array([40, 40, 32, 30, 40, 32, 40, 30])

if num_sessions == 0: num_sessions = nsessions_allsubj[subj-1]

num_samples_per_epoch = 12800 // num_devices 
# or if you want less samples per epoch if you're using less data, can do (750*num_sessions) // num_devices 

num_iterations_per_epoch = num_samples_per_epoch // batch_size

print("num_sessions =", num_sessions, "batch_size =", batch_size, "num_iterations_per_epoch =",num_iterations_per_epoch, "num_samples_per_epoch =",num_samples_per_epoch,)


# In[7]:


train_url = f"{data_path}/wds/subj0{subj}/train/" + "{0.." + f"{num_sessions-1}" + "}.tar"
print(train_url)

train_data = wds.WebDataset(train_url,resampled=True,nodesplitter=my_split_by_node)\
                    .shuffle(750, initial=1500, rng=random.Random(42))\
                    .decode("torch")\
                    .rename(behav="behav.npy", past_behav="past_behav.npy", future_behav="future_behav.npy", olds_behav="olds_behav.npy")\
                    .to_tuple(*["behav", "past_behav", "future_behav", "olds_behav"])\
                    .with_epoch(num_samples_per_epoch)
train_dl = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False, drop_last=True, pin_memory=True)

# if using same test dataset as MindEyeV1:
test_url = f"{data_path}/wds/subj0{subj}/test/" + "0.tar"
num_test = 2770 # total number of test samples
test_batch_size = 30 # number of test samples to use during validation
test_data = wds.WebDataset(test_url,resampled=False,nodesplitter=my_split_by_node)\
                    .shuffle(750, initial=1500, rng=random.Random(42))\
                    .decode("torch")\
                    .rename(behav="behav.npy", past_behav="past_behav.npy", future_behav="future_behav.npy", olds_behav="olds_behav.npy")\
                    .to_tuple(*["behav", "past_behav", "future_behav", "olds_behav"])
test_dl = torch.utils.data.DataLoader(test_data, batch_size=num_test, shuffle=False, drop_last=False, pin_memory=True)

num_sessions = 3 batch_size = 8 num_iterations_per_epoch = 533 num_samples_per_epoch = 4266
/weka/proj-fmri/shared/mindeyev2_dataset/wds/subj01/train/{0..2}.tar


In [19]:
f = h5py.File(f'{data_path}/betas_all_subj0{subj}_fp32.hdf5', 'r')

voxels = f['betas'][:]
print(f"subj0{subj} betas loaded into memory")
voxels = torch.Tensor(voxels).to("cpu").to(data_type)
print("voxels", voxels.shape)
num_voxels = voxels.shape[-1]

# # load orig images
f = h5py.File(f'{data_path}/coco_images_224_float16.hdf5', 'r')
images = f['images'][:]
images = torch.Tensor(images).to("cpu").to(data_type)
print("images", images.shape)

subj01 betas loaded into memory
voxels torch.Size([30000, 15724])
images torch.Size([73000, 3, 224, 224])


In [45]:
class imgViTBG(Dataset):
    def __init__(self, directory):
        self.directory = directory
        
    def __len__(self):
        return len(os.listdir(self.directory))
    
    def __getitem__(self, idx):
        """
        Load and return the tensor at the given index.
        Args:
        - idx (int): Index of the tensor to be loaded.
        """
        filename = f"imgt{idx+1}_embed.pt"
        file_path = os.path.join(self.directory, filename)
        # tensor = torch.load(file_path)
        return file_path
directory = '/weka/proj-fmri/shared/vitBG_embeds/'  # Replace with your directory path
imgemb_dataset = imgViTBG(directory)

In [91]:
for train_i, (behav, past_behav, future_behav, old_behav) in enumerate(train_dl):
    if train_i == 8:
        voxel = voxels[behav[:,0,5].cpu().long()].to(d1)
        image = images[behav[:,0,0].cpu().long()].to(d1).float()
        arr = behav[:,0,0].cpu().long()
        print(behav[:,0,0].cpu().long())


tensor([36418, 70462, 47512, 69134, 11638, 69462, 58506, 59255])


In [92]:
arr

tensor([36418, 70462, 47512, 69134, 11638, 69462, 58506, 59255])

In [98]:
image.shape

torch.Size([8, 3, 224, 224])

In [21]:
clip_img_embedder = FrozenOpenCLIPImageEmbedder(
    arch="ViT-bigG-14",
    version="laion2b_s39b_b160k",
    output_tokens=True,
    only_tokens=True,
).to(data_type)
clip_img_embedder.to(d1)

FrozenOpenCLIPImageEmbedder(
  (model): CLIP(
    (visual): VisionTransformer(
      (conv1): Conv2d(3, 1664, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (patch_dropout): Identity()
      (ln_pre): LayerNorm((1664,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): ModuleList(
          (0-47): 48 x ResidualAttentionBlock(
            (ln_1): LayerNorm((1664,), eps=1e-05, elementwise_affine=True)
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=1664, out_features=1664, bias=True)
            )
            (ls_1): Identity()
            (ln_2): LayerNorm((1664,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
              (c_fc): Linear(in_features=1664, out_features=8192, bias=True)
              (gelu): GELU(approximate='none')
              (c_proj): Linear(in_features=8192, out_features=1664, bias=True)
            )
            (ls_2): Identity()


In [44]:
for i in tqdm(range(73000)):
    img = images[i].unsqueeze(0).to(d1)
    emb = clip_img_embedder(img).to(data_type)
    full_path = os.path.join("/weka/proj-fmri/shared/vitBG_embeds/", f"imgt{i+1}_embed.pt")
    torch.save(emb, full_path)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 73000/73000 [33:52<00:00, 35.91it/s]


In [33]:
images.dtype

torch.float32

In [85]:
arr = [65000, 65001, 2, 3]

In [93]:
def get_img_tensor(data, index_arr, batch_size):
    emb_arr = []
    for i in range(batch_size):
        ind = index_arr[i]
        path_emb = data[ind]
        emb = torch.load(path_emb, map_location='cpu')
        emb = emb.squeeze(0)
        emb_arr.append(emb)
    emb_tensor = torch.stack(emb_arr)
    return emb_tensor
x = get_img_tensor(imgemb_dataset, arr, 8).cuda()
x = x.to(torch.float16)
print(x.dtype)

/weka/proj-fmri/shared/vitBG_embeds/imgt36419_embed.pt
/weka/proj-fmri/shared/vitBG_embeds/imgt70463_embed.pt
/weka/proj-fmri/shared/vitBG_embeds/imgt47513_embed.pt
/weka/proj-fmri/shared/vitBG_embeds/imgt69135_embed.pt
/weka/proj-fmri/shared/vitBG_embeds/imgt11639_embed.pt
/weka/proj-fmri/shared/vitBG_embeds/imgt69463_embed.pt
/weka/proj-fmri/shared/vitBG_embeds/imgt58507_embed.pt
/weka/proj-fmri/shared/vitBG_embeds/imgt59256_embed.pt
torch.float16


In [94]:
gt_image = images[arr].cuda()
z = clip_img_embedder(gt_image)
print(z.shape)
z.dtype

torch.Size([8, 256, 1664])


torch.float16

In [98]:
x[3]

tensor([[ 1.1475e+00, -8.7012e-01,  4.7339e-01,  ..., -7.4268e-01,
          1.1846e+00,  4.8267e-01],
        [ 9.2725e-01, -3.5107e-01,  7.5388e-04,  ...,  4.9316e-01,
          9.3872e-02,  1.7566e-01],
        [ 8.7891e-01, -1.0576e+00, -3.3325e-01,  ..., -8.5596e-01,
         -2.2253e-01,  4.0649e-01],
        ...,
        [ 1.9958e-01,  1.3599e-01, -3.6938e-01,  ...,  7.1338e-01,
          7.2852e-01,  1.0244e+00],
        [-1.0602e-01, -7.0898e-01, -2.7008e-02,  ..., -7.0752e-01,
          1.0225e+00,  7.6025e-01],
        [ 2.0483e-01,  2.2217e-01, -5.3223e-01,  ..., -6.9141e-01,
          6.4111e-01, -9.6289e-01]], device='cuda:0', dtype=torch.float16)

In [97]:
z[3]

tensor([[ 1.1504, -0.8696,  0.4709,  ..., -0.7407,  1.1875,  0.4797],
        [ 0.9287, -0.3550,  0.0100,  ...,  0.4846,  0.1002,  0.1641],
        [ 0.8770, -1.0576, -0.3286,  ..., -0.8540, -0.2194,  0.4082],
        ...,
        [ 0.1993,  0.1409, -0.3718,  ...,  0.7129,  0.7329,  1.0283],
        [-0.1082, -0.7070, -0.0278,  ..., -0.7041,  1.0215,  0.7622],
        [ 0.2048,  0.2233, -0.5347,  ..., -0.6909,  0.6411, -0.9604]],
       device='cuda:0', dtype=torch.float16)